In [2]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import pickle

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 700)

This notebook processes data for ADD/ADH calculation using script:
* create_weather_related_features_parallelized.py

In [23]:
# import data
#data = pd.read_pickle('../data/Gelderman_SOD_cohort/for_pmi_estimation.pkl')[['new_id','donor_date', 'correct_img_date',
       #'date_placed_ARF', 'PMI_days']]
    
#data = pd.read_csv('../data/Gelderman_SOD_cohort/unique_img_triplets_plus.csv')[['new_id','donor_date', 'correct_img_date',
       #'date_placed_ARF', 'PMI_days']]
data = pd.read_pickle('../data/master_dataset.pkl')[['new_id', 'correct_img_date', 'img',
       'date_placed_ARF', 'PMI_days']]
print(data.shape)
display(data.head())
display(data.info())

(1132970, 5)


,new_id,correct_img_date,img,date_placed_ARF,PMI_days
0,000,2018-01-21,00000121.01.JPG,2018-01-21,0.0
1,000,2018-01-21,00000121.02.JPG,2018-01-21,0.0
2,000,2018-01-21,00000121.03.JPG,2018-01-21,0.0
3,000,2018-01-21,00000121.04.JPG,2018-01-21,0.0
4,000,2018-01-21,00000121.05.JPG,2018-01-21,0.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1132970 entries, 0 to 1132969
Data columns (total 5 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   new_id            1132970 non-null  object 
 1   correct_img_date  1132970 non-null  object 
 2   img               1132970 non-null  object 
 3   date_placed_ARF   1132970 non-null  object 
 4   PMI_days          1132970 non-null  float64
dtypes: float64(1), object(4)
memory usage: 51.9+ MB


None

In [24]:
# extract donor_date from img column
data['donor_date'] = data.img.str.split('.', expand=True)[0]
display(data.sample(3))

,new_id,correct_img_date,img,date_placed_ARF,PMI_days,donor_date
584753,7ec,2017-02-15,7ec00215.21.JPG,2017-02-09,6.0,7ec00215
498837,65b,2013-10-26,65b01026.49.JPG,2013-09-27,29.0,65b01026
612910,84c,2016-11-03,84c01103.25.JPG,2016-11-01,2.0,84c01103


In [25]:
data['date_placed_ARF'] = pd.to_datetime(data['date_placed_ARF'], format='%Y-%m-%d')
data['correct_img_date'] = pd.to_datetime(data['correct_img_date'],format='%Y-%m-%d')
display(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1132970 entries, 0 to 1132969
Data columns (total 6 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   new_id            1132970 non-null  object        
 1   correct_img_date  1132970 non-null  datetime64[ns]
 2   img               1132970 non-null  object        
 3   date_placed_ARF   1132970 non-null  datetime64[ns]
 4   PMI_days          1132970 non-null  float64       
 5   donor_date        1132970 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(3)
memory usage: 60.5+ MB


None

In [26]:
data.sort_values('donor_date').head(10)

,new_id,correct_img_date,img,date_placed_ARF,PMI_days,donor_date
0,000,2018-01-21,00000121.01.JPG,2018-01-21,0.0,00000121
1,000,2018-01-21,00000121.02.JPG,2018-01-21,0.0,00000121
2,000,2018-01-21,00000121.03.JPG,2018-01-21,0.0,00000121
3,000,2018-01-21,00000121.04.JPG,2018-01-21,0.0,00000121
4,000,2018-01-21,00000121.05.JPG,2018-01-21,0.0,00000121
5,000,2018-01-21,00000121.06.JPG,2018-01-21,0.0,00000121
6,000,2018-01-21,00000121.07.JPG,2018-01-21,0.0,00000121
7,000,2018-01-21,00000121.08.JPG,2018-01-21,0.0,00000121
8,000,2018-01-21,00000121.09.JPG,2018-01-21,0.0,00000121
24,000,2018-01-22,00000122.16.JPG,2018-01-21,1.0,00000122


In [6]:
unique_donor_date = data.drop_duplicates('donor_date').copy()
print(len(unique_donor_date))
display(unique_donor_date.head())
display(unique_donor_date.info())

38156


,new_id,correct_img_date,img,date_placed_ARF,PMI_days,donor_date
0,000,2018-01-21,00000121.01.JPG,2018-01-21,0.0,00000121
9,000,2018-01-22,00000122.01.JPG,2018-01-21,1.0,00000122
34,000,2018-01-23,00000123.01.JPG,2018-01-21,2.0,00000123
60,000,2018-01-24,00000124.01.JPG,2018-01-21,3.0,00000124
116,000,2018-01-25,00000125.01.JPG,2018-01-21,4.0,00000125


<class 'pandas.core.frame.DataFrame'>
Int64Index: 38156 entries, 0 to 1132967
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   new_id            38156 non-null  object        
 1   correct_img_date  38156 non-null  datetime64[ns]
 2   img               38156 non-null  object        
 3   date_placed_ARF   38156 non-null  datetime64[ns]
 4   PMI_days          38156 non-null  float64       
 5   donor_date        38156 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(3)
memory usage: 2.0+ MB


None

In [8]:
unique_donor_date[unique_donor_date.donor_date == '27000516']

,new_id,correct_img_date,img,date_placed_ARF,PMI_days,donor_date
336376,270,2015-05-16,27000516.01.JPG,2015-05-12,4.0,27000516


In [12]:
# save unique_donor_date which will be used to calculate ADD
#unique_donor_date.to_pickle('../data/Gelderman_SOD_cohort/unique_donor_date.pkl')

## After running create_weather_related_features_parallelized.py:

In [3]:
# import ADD data
with open('./../data/unique_donor_date_w_ADD.pkl', 'rb') as f:
        add_data = pickle.load(f)
add_data = pd.DataFrame.from_dict(add_data)
print(add_data.shape)
display(add_data.head())
display(add_data.info())
display(add_data.describe())

(38156, 14)


,new_id,old_id,img,correct_img_date,date_placed_ARF,PMI_days,donor_date,ADD_thres0,ADD_thres5,ADD_thres10,ADD_thres15,ADD_thres20,ADD_thres25,ADD_thres30
0,000,UT01-18D,00000121.01.JPG,2018-01-21,2018-01-21,0.0,00000121,8.645833,8.645833,0.000000,0.0,0.0,0.0,0.0
1,000,UT01-18D,00000122.01.JPG,2018-01-22,2018-01-21,1.0,00000122,19.072917,19.072917,10.427083,0.0,0.0,0.0,0.0
2,000,UT01-18D,00000123.01.JPG,2018-01-23,2018-01-21,2.0,00000123,28.018750,28.018750,10.427083,0.0,0.0,0.0,0.0
3,000,UT01-18D,00000124.01.JPG,2018-01-24,2018-01-21,3.0,00000124,31.940278,28.018750,10.427083,0.0,0.0,0.0,0.0
4,000,UT01-18D,00000125.01.JPG,2018-01-25,2018-01-21,4.0,00000125,35.281944,28.018750,10.427083,0.0,0.0,0.0,0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38156 entries, 0 to 38155
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   new_id            38156 non-null  object        
 1   old_id            38156 non-null  object        
 2   img               38156 non-null  object        
 3   correct_img_date  38156 non-null  datetime64[ns]
 4   date_placed_ARF   38156 non-null  datetime64[ns]
 5   PMI_days          38156 non-null  float64       
 6   donor_date        38156 non-null  object        
 7   ADD_thres0        38156 non-null  float64       
 8   ADD_thres5        38156 non-null  float64       
 9   ADD_thres10       38156 non-null  float64       
 10  ADD_thres15       38156 non-null  float64       
 11  ADD_thres20       38156 non-null  float64       
 12  ADD_thres25       38156 non-null  float64       
 13  ADD_thres30       38156 non-null  float64       
dtypes: datetime64[ns](2), 

None

,PMI_days,ADD_thres0,ADD_thres5,ADD_thres10,ADD_thres15,ADD_thres20,ADD_thres25,ADD_thres30
count,38156.000000,38156.000000,38156.000000,38156.000000,38156.000000,38156.000000,38156.000000,38156.000000
mean,108.044187,1656.323011,1622.277232,1493.589792,1267.674602,985.241439,410.107323,1.788047
std,179.790425,2894.725215,2848.195614,2646.469322,2307.194925,1918.923059,879.616410,10.450382
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,21.000000,269.428864,253.585208,192.245402,99.605208,0.000000,0.000000,0.000000
50%,47.000000,613.335718,591.026898,511.135476,369.416736,170.238750,0.000000,0.000000
75%,101.000000,1468.078194,1449.167801,1376.629306,1212.954431,985.056361,342.772536,0.000000
max,2359.000000,37258.976342,36590.407941,34249.049739,29879.681015,23743.136810,8599.246237,126.045139


In [4]:
# join data with add_data
data = pd.read_pickle('../data/master_dataset.pkl')
data['donor_date'] = data.img.str.split('.', expand=True)[0]

data2 = pd.merge(data, add_data[['donor_date', 'ADD_thres0', 'ADD_thres5', 'ADD_thres10', 'ADD_thres15',
       'ADD_thres20', 'ADD_thres25', 'ADD_thres30']], how='inner', on='donor_date')
print(len(data2))
display(data2.head())
display(data2.info())

1132970


,new_id,old_id,img_path,img,correct_img_date,date_placed_ARF,PMI_days,year,sex,ancestry,est_stature_cm,est_weight_lb,est_stature_in,age_at_death,true_BP,pred_BP,pred_BP_conf,true_SOD_G,BP_of_true_SOD_G,pred_SOD_G,pred_SOD_G_conf,true_SOD_M,BP_of_true_SOD_M,donor_date,ADD_thres0,ADD_thres5,ADD_thres10,ADD_thres15,ADD_thres20,ADD_thres25,ADD_thres30
0,000,UT01-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.01.JPG,00000121.01.JPG,2018-01-21,2018-01-21,0.0,2018.0,Male,NaN,177.8,160.0,70.000038,64.0,NaN,stake,100.00,NaN,NaN,NaN,NaN,NaN,NaN,00000121,8.645833,8.645833,0.0,0.0,0.0,0.0,0.0
1,000,UT01-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.02.JPG,00000121.02.JPG,2018-01-21,2018-01-21,0.0,2018.0,Male,NaN,177.8,160.0,70.000038,64.0,NaN,fullbody,85.51,NaN,NaN,NaN,NaN,NaN,NaN,00000121,8.645833,8.645833,0.0,0.0,0.0,0.0,0.0
2,000,UT01-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.03.JPG,00000121.03.JPG,2018-01-21,2018-01-21,0.0,2018.0,Male,NaN,177.8,160.0,70.000038,64.0,NaN,fullbody,90.52,NaN,NaN,NaN,NaN,NaN,NaN,00000121,8.645833,8.645833,0.0,0.0,0.0,0.0,0.0
3,000,UT01-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.04.JPG,00000121.04.JPG,2018-01-21,2018-01-21,0.0,2018.0,Male,NaN,177.8,160.0,70.000038,64.0,NaN,fullbody,99.50,NaN,NaN,NaN,NaN,NaN,NaN,00000121,8.645833,8.645833,0.0,0.0,0.0,0.0,0.0
4,000,UT01-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000121.05.JPG,00000121.05.JPG,2018-01-21,2018-01-21,0.0,2018.0,Male,NaN,177.8,160.0,70.000038,64.0,NaN,fullbody,43.47,NaN,NaN,NaN,NaN,NaN,NaN,00000121,8.645833,8.645833,0.0,0.0,0.0,0.0,0.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1132970 entries, 0 to 1132969
Data columns (total 31 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   new_id            1132970 non-null  object 
 1   old_id            1132970 non-null  object 
 2   img_path          1132970 non-null  object 
 3   img               1132970 non-null  object 
 4   correct_img_date  1132970 non-null  object 
 5   date_placed_ARF   1132970 non-null  object 
 6   PMI_days          1132970 non-null  float64
 7   year              1125446 non-null  float64
 8   sex               1125106 non-null  object 
 9   ancestry          222775 non-null   object 
 10  est_stature_cm    878776 non-null   float64
 11  est_weight_lb     1052284 non-null  float64
 12  est_stature_in    1086628 non-null  float64
 13  age_at_death      1125128 non-null  float64
 14  true_BP           45930 non-null    object 
 15  pred_BP           1132692 non-null  object 
 16  

None

In [7]:
data2.to_csv('../data/master_dataset_w_ADD.csv', index_label=False)